In [1]:
from rna_resnet import ResNet
from keras import callbacks as cb
from Calibration_Util import FileIO as io
import os
import numpy as np
import CostFunctions as cf
from keras import backend as K
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from plots import heatmap
from sys import argv

Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
folder = '../data/6pc/'
# GTEX as source and TCGA as target
source_file = 'unnorm-log-6PC-GTEX-breast-prostate-thyroid.csv'
target_file = 'unnorm-log-6PC-TCGA-breast-prostate-thyroid.csv'
    
source_path = folder + source_file
target_path = folder + target_file

In [5]:
rnaNet0 = ResNet(layer_sizes=[6, 20])
rnaNet0.load_data(source_path=source_path, target_path=target_path)
print("gtex = source shape = " + str(rnaNet0.source.shape))
print("tcga = target shape = " + str(rnaNet0.target.shape))
rnaNet0.init_res_net()
callbacks=[rnaNet0.lrate, cb.EarlyStopping(monitor='val_loss', patience=100, mode='auto')]
rnaNet0.train(epochs=1000, callbacks=callbacks, batch_size=50)
print("finished training")

print("Running sanity check...")
rnaNet0.predict()
source = rnaNet0.source.astype('float32')
target = rnaNet0.target.astype('float32')
calibrated_source = rnaNet0.calibrated_source.astype('float32')

mmd = cf.MMD(source, target, MMDTargetSampleSize=target.shape[0], n_neighbors=10)
mmd_before = K.eval(mmd.cost(source, target))
mmd_after = K.eval(mmd.cost(calibrated_source, target))

print("MMD before: %0.10f" % mmd_before)
print("MMD after: %0.10f" % mmd_after)

gtex = source shape = (636, 6)
tcga = target shape = (211, 6)
setting scales using KNN
[12.431486575826753, 24.862973151653506, 49.72594630330701]
setting all scale weights to 1
Train on 572 samples, validate on 64 samples
Epoch 1/1000
572/572 [==============================] - 1s 874us/step - loss: 1.2328 - val_loss: 1.8634
Epoch 2/1000
572/572 [==============================] - 0s 244us/step - loss: 1.2051 - val_loss: 1.8057
Epoch 3/1000
572/572 [==============================] - 0s 243us/step - loss: 1.1424 - val_loss: 1.7467
Epoch 4/1000
572/572 [==============================] - 0s 243us/step - loss: 1.0384 - val_loss: 1.6481
Epoch 5/1000
572/572 [==============================] - 0s 244us/step - loss: 0.9556 - val_loss: 1.6185
Epoch 6/1000
572/572 [==============================] - 0s 240us/step - loss: 0.9214 - val_loss: 1.5949
Epoch 7/1000
572/572 [==============================] - 0s 237us/step - loss: 0.9489 - val_loss: 1.5110
Epoch 8/1000
572/572 [===========================

Epoch 77/1000
572/572 [==============================] - 0s 246us/step - loss: 0.8322 - val_loss: 1.4716
Epoch 78/1000
572/572 [==============================] - 0s 241us/step - loss: 0.8181 - val_loss: 1.5047
Epoch 79/1000
572/572 [==============================] - 0s 249us/step - loss: 0.8368 - val_loss: 1.5287
Epoch 80/1000
572/572 [==============================] - 0s 242us/step - loss: 0.8352 - val_loss: 1.5170
Epoch 81/1000
572/572 [==============================] - 0s 244us/step - loss: 0.8372 - val_loss: 1.5681
Epoch 82/1000
572/572 [==============================] - 0s 240us/step - loss: 0.8314 - val_loss: 1.3395
Epoch 83/1000
572/572 [==============================] - 0s 243us/step - loss: 0.8436 - val_loss: 1.4369
Epoch 84/1000
572/572 [==============================] - 0s 248us/step - loss: 0.8171 - val_loss: 1.5140
Epoch 85/1000
572/572 [==============================] - 0s 241us/step - loss: 0.8432 - val_loss: 1.5214
Epoch 86/1000
572/572 [==============================] 

572/572 [==============================] - 0s 246us/step - loss: 0.8044 - val_loss: 1.4321
Epoch 155/1000
572/572 [==============================] - 0s 245us/step - loss: 0.8360 - val_loss: 1.5092
Epoch 156/1000
572/572 [==============================] - 0s 244us/step - loss: 0.8314 - val_loss: 1.5004
Epoch 157/1000
572/572 [==============================] - 0s 244us/step - loss: 0.8060 - val_loss: 1.3636
Epoch 158/1000
572/572 [==============================] - 0s 242us/step - loss: 0.8012 - val_loss: 1.5028
Epoch 159/1000
572/572 [==============================] - 0s 239us/step - loss: 0.8110 - val_loss: 1.6082
Epoch 160/1000
572/572 [==============================] - 0s 240us/step - loss: 0.8195 - val_loss: 1.4616
Epoch 161/1000
572/572 [==============================] - 0s 244us/step - loss: 0.8052 - val_loss: 1.3581
Epoch 162/1000
572/572 [==============================] - 0s 239us/step - loss: 0.8524 - val_loss: 1.3465
Epoch 163/1000
572/572 [==============================] - 0s 

In [ ]:
rnaNet1 = ResNet(layer_sizes=[6, 20])
rnaNet1.load_data(source_path=source_path, target_path=target_path)
print("gtex = source shape = " + str(rnaNet1.source.shape))
print("tcga = target shape = " + str(rnaNet1.target.shape))
rnaNet1.init_res_net()
callbacks=[rnaNet1.lrate, cb.EarlyStopping(monitor='val_loss', patience=100, mode='auto')]
rnaNet1.train(epochs=1000, callbacks=callbacks, batch_size=50)
print("finished training")

print("Running sanity check...")
rnaNet1.predict()
source = rnaNet1.source.astype('float32')
target = rnaNet1.target.astype('float32')
calibrated_source = rnaNet1.calibrated_source.astype('float32')

mmd = cf.MMD(source, target, MMDTargetSampleSize=target.shape[0], n_neighbors=10)
mmd_before = K.eval(mmd.cost(source, target))
mmd_after = K.eval(mmd.cost(calibrated_source, target))

print("MMD before: %0.10f" % mmd_before)
print("MMD after: %0.10f" % mmd_after)

gtex = source shape = (636, 6)
tcga = target shape = (211, 6)
setting scales using KNN
[12.253831663266027, 24.507663326532054, 49.01532665306411]
setting all scale weights to 1
Train on 572 samples, validate on 64 samples
Epoch 1/1000
572/572 [==============================] - 0s 720us/step - loss: 1.1704 - val_loss: 1.8165
Epoch 2/1000
572/572 [==============================] - 0s 244us/step - loss: 1.1389 - val_loss: 1.7456
Epoch 3/1000
572/572 [==============================] - 0s 248us/step - loss: 1.0941 - val_loss: 1.8538
Epoch 4/1000
572/572 [==============================] - 0s 251us/step - loss: 0.9994 - val_loss: 1.8929
Epoch 5/1000
572/572 [==============================] - 0s 245us/step - loss: 0.9225 - val_loss: 1.8890
Epoch 6/1000
572/572 [==============================] - 0s 245us/step - loss: 0.8833 - val_loss: 1.9174
Epoch 7/1000
572/572 [==============================] - 0s 247us/step - loss: 0.8608 - val_loss: 1.8778
Epoch 8/1000
572/572 [===========================

Epoch 77/1000
572/572 [==============================] - 0s 241us/step - loss: 0.7758 - val_loss: 1.4167
Epoch 78/1000
572/572 [==============================] - 0s 238us/step - loss: 0.7803 - val_loss: 1.5175
Epoch 79/1000
572/572 [==============================] - 0s 240us/step - loss: 0.7480 - val_loss: 1.8275
Epoch 80/1000
572/572 [==============================] - 0s 244us/step - loss: 0.7167 - val_loss: 1.6795
Epoch 81/1000
572/572 [==============================] - 0s 243us/step - loss: 0.7475 - val_loss: 1.5276
Epoch 82/1000
572/572 [==============================] - 0s 239us/step - loss: 0.7817 - val_loss: 1.6484
Epoch 83/1000
572/572 [==============================] - 0s 242us/step - loss: 0.7453 - val_loss: 1.3726
Epoch 84/1000
572/572 [==============================] - 0s 242us/step - loss: 0.7666 - val_loss: 1.5652
Epoch 85/1000
572/572 [==============================] - 0s 239us/step - loss: 0.7616 - val_loss: 1.8567
Epoch 86/1000
572/572 [==============================] 

In [ ]:
rnaNet0.predict()
source = rnaNet0.source.astype('float32')
target = rnaNet0.target.astype('float32')
calibrated_source = rnaNet0.calibrated_source.astype('float32')

mmd = cf.MMD(source, target, MMDTargetSampleSize=target.shape[0], n_neighbors=10)
mmd_before = K.eval(mmd.cost(source, target))
mmd_after = K.eval(mmd.cost(calibrated_source, target))

print("MMD before: %0.10f" % mmd_before)
print("MMD after: %0.10f" % mmd_after)

source = rnaNet1.source.astype('float32')
target = rnaNet1.target.astype('float32')
calibrated_source = rnaNet1.calibrated_source.astype('float32')

mmd = cf.MMD(source, target, MMDTargetSampleSize=target.shape[0], n_neighbors=10)
mmd_before = K.eval(mmd.cost(source, target))
mmd_after = K.eval(mmd.cost(calibrated_source, target))

print("MMD before: %0.10f" % mmd_before)
print("MMD after: %0.10f" % mmd_after)

rnaNet0.predict(data=source)
rnaNet1.predict(rnaNet0.calibrated_source)
calibrated_source = rnaNet1.calibrated_source

mmd = cf.MMD(source, target, MMDTargetSampleSize=target.shape[0], n_neighbors=10)
mmd_before = K.eval(mmd.cost(source, target))
mmd_after = K.eval(mmd.cost(calibrated_source, target))

print("MMD before: %0.10f" % mmd_before)
print("MMD after: %0.10f" % mmd_after)